In [1]:
import pandas as pd
import numpy as np
from os.path import join

In [2]:
dst = "../data/twitter/US_politician_twitter_accounts"

# Download data

In [3]:
# 114th congress (only senators!)
url = "https://www.socialseer.com/resources/us-senator-twitter-accounts-114th-congress-historical/"
df = pd.read_html(url)[0]
df.columns = df.loc[0]
df = df.drop(0)
df["congress"] = 114
df.to_csv(join(dst, "raw", "114_congress.csv"), index=False)

In [4]:
# 115th congress (only senators!)
url = "https://www.socialseer.com/resources/us-senator-twitter-accounts/"
df = pd.read_html(url)[0]
df.columns = df.loc[0]
df = df.drop(0)
df["congress"] = 115
df.to_csv(join(dst, "raw", "115_congress.csv"), index=False)

In [5]:
# 116 congress (senate and house) manually downloaded from 
# https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/MBOJNS
df_house = pd.read_csv(join(dst, "raw", "congress116-house-accounts.csv"))
df_senate = pd.read_csv(join(dst, "raw", "congress116-senate-accounts.csv"))
df_house["chamber"] = "house"
df_senate["chamber"] = "senate"
df = pd.concat([df_house, df_senate])
df["congress"] = 116
df.to_csv(join(dst, "raw", "116_congress.csv"), index=False)

In [6]:
# 117th congress (senate and house)
url = "https://triagecancer.org/congressional-social-media"
df = pd.read_html(url)[0]
df["congress"] = 117
df.to_csv(join(dst, "raw", "117_congress.csv"), index=False)

# Clean and merge data

In [8]:
df_114 = pd.read_csv(join(dst, "raw", "114_congress.csv"), 
                     usecols = ["Official Senator Twitter ID",
                                "Senate Staff Twitter Id",
                                "Campaign/Other ID", 
                                "congress"])
df_115 = pd.read_csv(join(dst, "raw", "115_congress.csv"),
                    usecols = ["Official Twitter", 
                               "Staff Twitter",
                               "Campaign Twitter",
                               "congress"])
df_116 = pd.read_csv(join(dst, "raw", "116_congress.csv"),
                    usecols=["Token", "congress"])
df_117 = pd.read_csv(join(dst, "raw", "117_congress.csv"),
                    usecols=["Twitter", "congress"])

In [9]:
df_114 = df_114.rename(columns={"Official Senator Twitter ID":"official",
                                "Senate Staff Twitter Id":"staff",
                                "Campaign/Other ID":"campaign"})
df_114 = pd.melt(df_114, value_vars=["official", "staff", "campaign"], 
        id_vars=["congress"], var_name="type", value_name="handle")

df_115 = df_115.rename(columns={"Official Twitter":"official",
                                "Staff Twitter":"staff",
                                "Campaign Twitter":"campaign"})
df_115 = pd.melt(df_115, value_vars=["official", "staff", "campaign"], 
        id_vars=["congress"], var_name="type", value_name="handle")

df_116 = df_116.rename(columns={"Token":"handle"})
df_116["type"] = "official"
df_117 = df_117.rename(columns={"Twitter":"handle"})
df_117["type"] = "official"

In [10]:
print(f"{len(df_114.dropna())} handles found for 114th congress")
print(f"{len(df_115.dropna())} handles found for 115th congress")
print(f"{len(df_116.dropna())} handles found for 116th congress")
print(f"{len(df_117.dropna())} handles found for 117th congress")

197 handles found for 114th congress
202 handles found for 115th congress
944 handles found for 116th congress
538 handles found for 117th congress


In [11]:
df = pd.concat([df_114, df_115, df_116, df_117])
df = df.dropna()
df["handle"] = df["handle"].apply(lambda x: x.replace("@", ""))

In [12]:
print(f"found a total of {len(df)} handles")
print(f"found a total of {len(df.handle.unique())} unique handles")

found a total of 1881 handles
found a total of 1143 unique handles


In [13]:
df.to_csv(join(dst, "clean", "congress-member-twitter-handles_114-117.csv"), index=False)
df[["handle"]]\
    .drop_duplicates()\
    .to_csv(join(dst, "clean", "congress-member-unique-twitter-handles_114-117.csv"), index=False)